In [9]:
import numpy as np
import pandas as pd
from pulp import *
from IPython.display import Image, display_png
%matplotlib inline

In [10]:
t = pd.read_csv('../csv/stochastic_matrix.csv', index_col=0).fillna(0)
t = t.T.drop([
    'C00001', # H2O
#     'C00003', # NAD
#     'C00004', # NADH
#     'C00005', # NADPH
#     'C00006', # NADP
    'C00009', # Phosphate
    'C00010', # CoA
#     'C00011', # CO2
    'C00033', # Acetate
    'C00080', # H+
    'C00138', # Reduced ferredoxin
    'C00139', # Oxidized ferredoxin
    'C15602',
    'C15603'
]).T

In [11]:
h = [
#     'R00112',
    'R00200',
    'R00268',
    'R00341',
    'R00342',
    'R00351',
    'R00361',
    'R00658',
    'R01015',
    'R01061',
    'R01070',
    'R01082',
    'R01196',
    'R01197',
    'R01325',
    'R01512',
    'R01518',
#     'R08549',
    'R01786',
    'R01899',
    'R01900',
    'R02164',
    'R02740',
    'R04779',
    'R10343'
]

In [12]:
p = t.T[h].T

for i in p:
    if sum(p[i] != 0) == 0:
        p=p.drop(i, axis=1)
        
p['C00007'] = pd.Series(np.zeros(p.shape[0]), index=p.index)
p = p.T.sort_index().T
p['Biomass'] = pd.Series(np.zeros(p.shape[0]), index=p.index)
p = p.T

x = [LpVariable(i) for i in p.index]
i = list(pd.read_csv('../csv/cnum_list.csv', index_col=0).index)
i.append('Biomass')
o = pd.DataFrame(x, index=i)

In [14]:
m = LpProblem(sense=LpMaximize)
m += o[0]['Biomass']

In [27]:
capacity_plus = 100
capacity_minus = -100

for i in p.columns:
    m += np.dot(list(p[i]), x) <= capacity_plus
    m += np.dot(list(p[i]), x) >= capacity_minus
    
# NADPH <-> NADH
m += o[0]['NADPH'] + o[0]['NAD+'] - o[0]['NADP+'] - o[0]['NADH'] <= capacity_plus
m += o[0]['NADPH'] + o[0]['NAD+'] - o[0]['NADP+'] - o[0]['NADH'] >= capacity_minus
    
# ATP <-> ADP
m += o[0]['ATP'] - o[0]['ADP'] <= capacity_plus
m += o[0]['ATP'] - o[0]['ADP'] >= capacity_minus

#  Oxophos
m += 2*o[0]['NADH'] + o[0]['Oxygen'] + 5*o[0]['ADP'] - 5*o[0]['ATP'] - 2*o[0]['NAD+'] <= capacity_plus
m += 2*o[0]['NADH'] + o[0]['Oxygen'] + 5*o[0]['ADP'] - 5*o[0]['ATP'] - 2*o[0]['NAD+'] >= capacity_minus

# biomass product # recon3Dから持ってくる
m += o[0]['Biomass'] + o[0]['ADP'] - o[0]['D-Glyceraldehyde 3-phosphate'] - o[0]['ATP'] - o[0]['3-Phospho-D-glycerate'] <= capacity_plus

m += o[0]['alpha-D-Glucos'] >= capacity_plus
m += o[0]['alpha-D-Glucos'] <= capacity_plus
m += o[0]['CO2'] <= 0
m += o[0]['CO2'] >= capacity_minus
m += o[0]['Oxygen'] >= 10
m += o[0]['Oxygen'] <= 10

In [28]:
status = m.solve()
print(LpStatus[status])

Optimal


In [32]:
o

,0
ATP,C00002
NAD+,C00003
NADH,C00004
NADPH,C00005
NADP+,C00006
Oxygen,C00007
ADP,C00008
CO2,C00011
Pyruvate,C00022
Acetyl-CoA,C00024


In [30]:
y = []
for i in x:
    y.append(value(i))

In [31]:
result = pd.DataFrame(y, index=o.index, columns=['value'])
result

,value
ATP,0.0
NAD+,200.0
NADH,0.0
NADPH,0.0
NADP+,100.0
Oxygen,10.0
ADP,98.0
CO2,0.0
Pyruvate,0.0
Acetyl-CoA,0.0
